In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


import matplotlib
from matplotlib import pyplot as plt
from textwrap import wrap


pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None  # default='warn'

import pickle
import time
from tqdm import tqdm
import re

from itertools import product
import collections

#import googletrans
#from googletrans import Translator
#to avoid instantiate too many translator, we created one instance for the whole notebook
#translator = Translator()


from prophet import Prophet
import itertools
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

import holidays


import hts.functions
import collections
from hts import HTSRegressor
from hts.hierarchy import HierarchyTree


from sktime.transformations.hierarchical.reconcile import Reconciler
from sktime.transformations.hierarchical.aggregate import Aggregator
from sktime.forecasting.exp_smoothing import ExponentialSmoothing


import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

# import data from pickle file

In [2]:
# read df_test_preds from pickle
pickle_dict_preds = pickle.load(open(f'../temp/df_test_preds.pkl', 'rb'))
df_test_preds = pickle_dict_preds['df_test_preds']
df_test_preds.head()

,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,item_name_tfidf_4,item_name_tfidf_5,item_name_tfidf_6,item_name_tfidf_7,item_name_tfidf_8,item_name_tfidf_9,cat_name_tfidf_0,cat_name_tfidf_1,cat_name_tfidf_2,cat_name_tfidf_3,cat_name_tfidf_4,cat_name_tfidf_5,cat_name_tfidf_6,cat_name_tfidf_7,cat_name_tfidf_8,cat_name_tfidf_9,cat_name_tfidf_10,cat_name_tfidf_11,cat_name_tfidf_12,cat_name_tfidf_13,cat_name_tfidf_14,prev_shopitem_sales_1,prev_shopitem_sales_2,prev_shopitem_sales_3,prev_shopitem_sales_4,prev_shopitem_sales_5,prev_shopitem_sales_6,prev_shopitem_sales_7,prev_shopitem_sales_8,prev_shopitem_sales_9,prev_shopitem_sales_10,prev_shopitem_sales_11,prev_shopitem_sales_12,prev_item_sales_1,prev_item_sales_2,prev_item_sales_3,prev_item_sales_4,prev_item_sales_5,prev_item_sales_6,prev_item_sales_7,prev_item_sales_8,prev_item_sales_9,prev_item_sales_10,prev_item_sales_11,prev_item_sales_12,shop_id,item_id,item_category_id,date_block_num,item_cnt_month
2731818,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2,5441,0,34,1.07
2731819,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,1.00,0.00,4.00,2.00,0.67,0.68,1.83,1.22,0.90,0.78,1.34,1.54,1.35,1.46,2.51,1.59,2,5643,2,34,0.37
2731820,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.69,0.00,0.73,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.67,0.68,0.61,0.20,0.00,0.00,0.00,0.00,0.03,0.02,0.05,0.05,2,13411,2,34,0.22
2731821,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.05,0.15,0.05,0.27,0.44,0.61,0.39,0.44,0.70,1.20,1.68,1.15,2,5634,2,34,0.12
2731822,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,2.00,0.00,0.00,0.02,0.07,0.10,0.24,0.20,0.34,0.34,0.41,0.65,0.66,1.15,0.00,2,5638,2,34,0.13


In [3]:
list_shop_id_filtered = [2, 3, 4, 5, 6, 7, 
                         10, 12, 14, 15, 16, 18, 19, 
                         21, 22, 24, 25, 26, 28, 
                         31, 34, 35, 37, 38, 39, 
                         41, 42, 44, 45, 46, 47, 48, 49, 
                         50, 52, 53, 55, 56, 57, 58, 59]




In [6]:
%%time

#initiate 'item_cnt_month_revised' value
df_test_preds['item_cnt_month_revised'] = df_test_preds['item_cnt_month']


#read forecasts of each shop and reconcile the forecasts with df_test_preds    
for shop_id in tqdm(list_shop_id_filtered):
    ############### read this shop data
    #################################################
    # read one shop data
    pickle_dict_oneshop = pickle.load(open(f'../temp/s{shop_id}_hier.pkl', 'rb'))
    df_hier  = pickle_dict_oneshop['df_hier']
    tree = pickle_dict_oneshop['tree']
    sum_mat = pickle_dict_oneshop['sum_mat']
    sum_mat_labels =pickle_dict_oneshop['sum_mat_labels']
    oneshop_cat_list = pickle_dict_oneshop['oneshop_cat_list']
    oneshop_item_list = pickle_dict_oneshop['oneshop_item_list']


    #read forecasts
    pickle_dict_oneshop_forecasts = pickle.load(open(f'../temp/s{shop_id}_forecasts.pkl', 'rb'))
    forecasts = pickle_dict_oneshop_forecasts['forecasts']



    ################### fill up all forecasts
    ###############################################
    for col in sum_mat_labels:
        if not(col in 'total' or col in oneshop_cat_list) :
            cat_item_id = col
            item_id = [int(s) for s in re.findall(r'\d+', cat_item_id)][1]

            fcst = df_test_preds.loc[(df_test_preds.shop_id == shop_id) & 
                                     ( df_test_preds.item_id == item_id )
                                     , ['item_cnt_month']
                                    ].values[0]
            forecasts[col] = list(fcst)
            #print(forecasts[col])

    ################# reconciliation
    ################################################
    pred_dict = collections.OrderedDict()
    for label in sum_mat_labels:
            pred_dict[label] = pd.DataFrame(data=forecasts[label].values, columns=['yhat'])        
    revised = hts.functions.optimal_combination(pred_dict, sum_mat, method='WLSS', mse={})
    revised_forecasts = pd.DataFrame(data=revised[0:,0:],
                                        index=forecasts.index,
                                        columns=sum_mat_labels)



    ############## update the revised forecast to df_test_preds
    ###################################################################
    dict_revised_forecasts = revised_forecasts.to_dict()
    for col in sum_mat_labels:
        if not(col in 'total' or col in oneshop_cat_list) :
            cat_item_id = col
            item_id = [int(s) for s in re.findall(r'\d+', cat_item_id)][1]

            df_test_preds.loc[(df_test_preds.shop_id == shop_id) & ( df_test_preds.item_id == item_id )
                             , 'item_cnt_month_revised'] = dict_revised_forecasts[cat_item_id]['fake']






100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [25:03<00:00, 36.66s/it]

Wall time: 25min 3s


#  Submission
- 'ols' score: 1.46928
- 'wlss' score: 1.11069

In [7]:
#get df_test from pickle
pickle_dict1 = pickle.load(open('../temp/data1.pkl', 'rb'))
df_test = pickle_dict1['df_test']

df_test_preds['item_cnt_month_revised'] = df_test_preds['item_cnt_month_revised'].clip(0,20)
df_submission = df_test_preds.merge(df_test, on=['shop_id', 'item_id'])[["ID", 'item_cnt_month_revised']]
df_submission.columns = ["ID", 'item_cnt_month']
df_submission.set_index('ID',drop=False, inplace=True)
df_submission.sort_index(inplace=True)

df_submission.to_csv("../temp/submission.csv", index=False)
df_submission.head()

,ID,item_cnt_month
ID,,
0,0,0.57
1,1,0.23
2,2,0.95
3,3,0.05
4,4,4.96
